1. NCC Algorithm with match window

2. SSD Algorithm

In [3]:
def dmap_ssd(left_img, right_img, kernel, max_offset):
    # Load in both images, assumed to be RGBA 8bit per channel images
    left_img = cv2.imread(left_img,cv2.IMREAD_GRAYSCALE)
    left = np.asarray(left_img)
    right_img = cv2.imread(right_img,cv2.IMREAD_GRAYSCALE)
    right = np.asarray(right_img)    
    height, width = left_img.shape # assume that both images are same size   
    # Depth (or disparity) map
    depth = np.zeros((width, height), np.uint8)
    depth.shape = height, width
       
    kernel_half = int(kernel / 2)    
    # offset_adjust = 255 / max_offset  # this is used to map depth map output to 0-255 range
      
    for y in range(kernel_half, height - kernel_half):      
        print(".", end="", flush=True)  # let the user know that something is happening (slowly!)
        
        for x in range(kernel_half, width - kernel_half):
            best_offset = 0
            prev_ssd = 65534

            print("\rCurrentHeight:{} CuurentWidth:{}".format(y,x),end="")
            
            for offset in range(max_offset):               
                ssd = 0
                ssd_temp = 0                            
                
                # v and u are the x,y of our local window search, used to ensure a good 
                # match- going by the squared differences of two pixels alone is insufficient, 
                # we want to go by the squared differences of the neighbouring pixels too
                for v in range(-kernel_half, kernel_half):
                    for u in range(-kernel_half, kernel_half):
                        # iteratively sum the sum of squared differences value for this block
                        # left[] and right[] are arrays of uint8, so converting them to int saves
                        # potential overflow, and executes a lot faster 
                        ssd_temp = int(left[y+v, x+u]) - int(right[y+v, (x+u) - offset])  
                        ssd += ssd_temp * ssd_temp              
                
                # if this value is smaller than the previous ssd at this block
                # then it's theoretically a closer match. Store this value against
                # this block..
                if ssd < prev_ssd:
                    prev_ssd = ssd
                    best_offset = offset
                            
            # set depth output for this x,y location to the best match
            depth[y, x] = best_offset #  * offset_adjust
                                
    # Convert to PIL and save it
    plt.subplots(figsize=(30, 30)) 
    plt.subplot(1,2,1)
    plt.imshow(Image.fromarray(depth), cmap='gray')
    plt.subplot(1,2,2)
    plt.imshow(left_img)
    return depth

disparityMap = dmap_ssd(left_images[0], right_images[0], 3, 69)

................

a. The rms (root mean squared) error between the values in your disparity map and those in the ground truth

In [2]:
groundTruth = cv2.imread(r'resources\Dataset\2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png')
def root_mean_squared_error(img1,img2):
    #判断两张图片的shape是否一致
    if img1.shape != img2.shape:
        print("images have different shape ")
    #计算两张图像的MSE
    img_rmse = np.sqrt(np.mean((img1.astype(np.float64) - img2.astype(np.float64))**2))
    #将图像的MSE量化为相似度
    img_sim = img_rmse / 255
    return img_rmse,1 - img_sim

root_mean_squared_error(groundTruth[:,:,0], disparityMap)

NameError: name 'disparityMap' is not defined

b. The fractions of pixels with errors less than 4, 2, 1, 0.5 and 0.25 pixels

In [199]:
def get_fraction(threshhold, imgTruth, img):
    TotalValidPixel = 0
    lessThan4 = 0
    lessThan2 = 0
    lessThan0_5 = 0
    lessThan0_25 = 0

    for y in range(len(imgTruth)):
        for x in range(len(imgTruth[0])):
            if imgTruth[y][x] != 0:
                TotalValidPixel+=1
                diff = abs(imgTruth[y][x] - img[y][x])
                if diff <= 0.25:
                    lessThan0_25 +=1 
                if diff <= 0.5:
                    lessThan0_5 += 1
                if diff <= 2:
                    lessThan2 += 1
                if diff <= 4:
                    lessThan4 +=1 
    print(lessThan4, lessThan2, lessThan0_5, lessThan0_25, TotalValidPixel)
    return lessThan4/TotalValidPixel, lessThan2/TotalValidPixel, lessThan0_5/TotalValidPixel, lessThan0_25/TotalValidPixel
            
get_fraction(1,groundTruth[:,:,0], disparityMap)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_17160\4186069593.py:12: RuntimeWarning: overflow encountered in ubyte_scalars
  diff = abs(imgTruth[y][x] - img[y][x])


399 241 76 76 88149


(0.004526426845454855,
 0.002734007192367469,
 0.000862176541991401,
 0.000862176541991401)